In [2]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler

In [3]:
#Load data
path = 'IMU/OxIOD/handheld'
train = [[1,7],[2,3],[3,5],[4,5]]
test = [[5,4]]

def read(locations):
    imu = []
    vicon = []
    for i in locations:
        j,k = i[0],i[1]
        l = 1
        while l <= k:
            ids = pd.read_csv(path+'/data'+str(j)+'/syn/imu'+str(l)+'.csv', header=None)
            vds = pd.read_csv(path+'/data'+str(j)+'/syn/vi'+str(l)+'.csv', header=None)
            imu.append(ids)
            vicon.append(vds)
            l = l + 1
    imu = pd.concat(imu)
    vicon = pd.concat(vicon)
    
    imu.drop(imu.columns[[0,1,2,3,7,8,9,13,14,15]],axis=1,inplace=True)
    extra = (len(imu)%200)
    imu = imu[:-extra]
    vicon.drop(vicon.columns[[0,1,4,5,6,7,8]],axis=1,inplace=True)

    vicon = vicon.rolling(200).mean()*2
    vicon = vicon[199::200]
    vicon['length'] = np.sqrt(vicon.iloc[:, 0]**2 + vicon.iloc[:,1]**2)
    vicon['totalheading'] = np.arctan(vicon.iloc[:,1]/vicon.iloc[:,0])
    vicon['heading'] = vicon['totalheading'].diff()

    vicon.drop(vicon.columns[[0,1,3]],axis=1,inplace=True)
    vicon.iloc[0,1] = vicon.iloc[1,1]
            
    sc = MinMaxScaler(feature_range = (0,1))
    imu = sc.fit_transform(imu)
    vicon = sc.fit_transform(vicon)
    
    return imu,vicon

imutrain, vicontrain = read(train)
imutest, vicontest = read(test)
print('done')

done


In [4]:
print(imutrain.shape)
print(vicontrain.shape)

def windowed_dataset(series, window_size, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 10, shift=10, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset
print(windowed_dataset(imutrain, 200, 256))

imutrain = imutrain.reshape(int(imutrain.shape[0]/200),200,imutrain.shape[1])

imutest = imutest.reshape(int(imutest.shape[0]/200),200,imutest.shape[1])


(699200, 6)
(3496, 2)
<PrefetchDataset shapes: (None, None, 6), types: tf.float64>


In [5]:
model = keras.Sequential()

model.add(layers.Bidirectional(layers.LSTM(96, input_shape=(200, 6))))

model.add(layers.Dense(2))

adam = keras.optimizers.Adam(lr=0.001)

model.compile(optimizer=adam,loss='mean_squared_error')

model.fit(imutrain,vicontrain, epochs=10, batch_size=256)

model.summary()

Epoch 1/10
14/14 [==============================] - 5s 343ms/step - loss: 0.0804
Epoch 2/10
14/14 [==============================] - 5s 341ms/step - loss: 0.0500
Epoch 3/10
14/14 [==============================] - 5s 348ms/step - loss: 0.0469
Epoch 4/10
14/14 [==============================] - 5s 340ms/step - loss: 0.0459
Epoch 5/10
14/14 [==============================] - 5s 359ms/step - loss: 0.0452
Epoch 6/10
14/14 [==============================] - 5s 336ms/step - loss: 0.0454
Epoch 7/10
14/14 [==============================] - 5s 337ms/step - loss: 0.0454
Epoch 8/10
14/14 [==============================] - 5s 332ms/step - loss: 0.0441
Epoch 9/10
14/14 [==============================] - 5s 324ms/step - loss: 0.0436
Epoch 10/10
14/14 [==============================] - 5s 337ms/step - loss: 0.0432
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 1

In [123]:
values = model.predict(imutest)
print(values)
print(vicontest)

[[0.49496493 0.5220183 ]
 [0.49568763 0.52089596]
 [0.49751118 0.5103168 ]
 ...
 [0.58469254 0.5478025 ]
 [0.4843044  0.59225005]
 [0.75763875 0.24322045]]
[[0.53372739 0.4972455 ]
 [0.53637204 0.4972455 ]
 [0.53392283 0.49740099]
 ...
 [0.47356541 0.64506133]
 [0.51825609 0.7023108 ]
 [0.68384809 0.08972516]]
